In [3]:
import random as r;
import numpy as np;
import copy as c;
from google.colab import files;
import pandas as pd;
from multiprocessing.pool import ThreadPool;
import multiprocessing as mp;

In [6]:
def element_is_found(arr, el):
  for i in range(0,len(arr)):
    if(el == arr[i]):
      return True;
  return False;

In [8]:
def generate_genes(state_data):
  arr = [];
  sample = list(range(0,state_data[2].shape[0]));

  for i in range(0,state_data[0].shape[0]):
    el = np.random.choice(sample);
    arr.append(el);
    sample.remove(el);

  return arr;

In [9]:
def model(params, state_data, chromosomes, is_test):
  
  (population, generations, crossover, mutation, alive) = params
  #get dimension size and use it to count chromosome elements
  genes_count = state_data[0].shape[0]

  for m in range(0,generations):
   chromosomes.sort(key=lambda a: a[1]);
   if m == generations-1 and not is_test:
     return chromosomes[0:alive];
   elif m == generations-1 and is_test:
      return chromosomes
   chromosomes = chromosomes[0:alive];

   new_chromosomes = [];
   for j in range (0, int(population - alive)):    
     child_gen = np.zeros((genes_count), dtype=int);

     gene_range = list(range(0,genes_count));
     father = chromosomes[np.random.choice(list(range(0,alive)))];
     mother = chromosomes[np.random.choice(list(range(0,alive)))];
     
     father_gen = c.deepcopy(father[0]);
     mother_gen = c.deepcopy(mother[0]);

     for k in range(0,crossover):
       cross_gen = np.random.choice(gene_range);
       child_gen[cross_gen] = father_gen[cross_gen];
       mother_gen[cross_gen] = 0;
       gene_range.remove(cross_gen);

     child_gen = np.add(mother_gen,child_gen);

     for i in range(0,mutation):
      should_mutate = bool(r.getrandbits(1))
    
      if should_mutate:
       gene_range_mutate = list(range(0,genes_count));
      
       cross_gen_first = np.random.choice(gene_range_mutate);
       gene_range_mutate.remove(cross_gen_first);
       cross_gen_second = np.random.choice(gene_range_mutate);
      
       placeholder = child_gen[cross_gen_first];
       child_gen[cross_gen_first] = child_gen[cross_gen_second];
       child_gen[cross_gen_second] = placeholder;

     child = create_chromosome(generate_chromosome, child_gen, state_data);
     new_chromosomes.append(child);
   chromosomes = [*chromosomes, *new_chromosomes]
  return chromosomes;

In [10]:
def create_chromosome(gen_chr, genes, state_data): 
  if gen_chr != None:
    return gen_chr(genes, state_data);
  return [];  

In [11]:
def generate_chromosome(genes, state_data):
    (adjacent_nodes, binary_tuple_digits, hamming_distances) = state_data;
    fitness_value = 0;
    chromosome_genes = c.deepcopy(genes);
    if len(set(chromosome_genes)) != len(chromosome_genes):
        fitness_value = 100000;
    if fitness_value == 0:
      for i in range(0,len(chromosome_genes)):
        for j in range(0,len(chromosome_genes)):
          fitness_value = fitness_value + adjacent_nodes[i][j] * hamming_distances[chromosome_genes[i]][chromosome_genes[j]];
    return (np.array(chromosome_genes), fitness_value);

In [12]:
def generate_population(population, state_data):
 chromosomes = [];
 for j in range(0,population):
    chr = create_chromosome(generate_chromosome, generate_genes(state_data), state_data);
    chromosomes.append(chr);
 return chromosomes;

In [13]:
def result_to_csv(results):
 data, adjacency_graph = results ;
 binary_list = [];
 labels = []

 for i in range(0,len(data)):
   labels.append("a" + str(i) + "=");
   binary_list.append(bin(data[i])[2:].zfill(state_generator(adjacency_graph)[1]));
 a1 = np.array(labels)
 a2 = np.array(data)
 a3 = np.array(binary_list)
 a1_column = np.reshape(a1, (-1, 1))
 a2_column = np.reshape(a2, (-1, 1))
 a3_column = np.reshape(a3, (-1, 1))

 matrix = np.concatenate((a1_column, a2_column, a3_column), axis=1)
 df = pd.DataFrame(matrix, columns=None)
 styled_df = df.style.hide_index().hide_columns()

 display(styled_df)

 #styled_df.to_excel('results.xlsx', index=False)
 #files.download('results.xlsx')

In [14]:
def list_binary_rep(n):
  seed = ""
  end = ""
  result = []
  
  for i in range(0,n):
    seed = seed + "0";
    end = end + "1"
  
  state = seed
  old_state = state
  while state != end:
    if len(result) != 0 :
     for i in range(0,len(result)):
      old_state = result[i]
      for i in range(0,n):
       if state[i] != "1":
        state = old_state[:i] + "1" + old_state[i + 1:]
        if state not in result:
          result.append(state)
   
    for i in range(0,n):
      if state[i] != "1":
       state = old_state[:i] + "1" + old_state[i + 1:]
       if state not in result:
         result.append(state)

  result.insert(0,seed);
  return result;

In [15]:
def generate_hamming_code(codes):
   array_zeros = np.zeros((len(codes), len(codes)));

   for i in range(0,len(codes)):
     for j in range(len(codes)-1, -1, -1):
       array_zeros[i][j] = calc_diff(codes[i],codes[j]);
   return array_zeros;

In [16]:
def calc_diff(stp, stt):
 el = 0;
 for i in range(0,len(stp)):
   if(stp[i] != stt[i]):
     el = el + 1;   
 return el; 

In [17]:
def find_min_constrain(value, power):
   if value <= np.power(2, power):
     return power
   return find_min_constrain(value, power + 1)  

In [18]:
def state_generator(adjacency_graph):
  binary_tuple_digits = find_min_constrain(adjacent_nodes.shape[0], 0);                          
  hamming_distances = generate_hamming_code(list_binary_rep(binary_tuple_digits));
  return (adjacent_nodes, binary_tuple_digits, hamming_distances)

In [1]:
def simulation(population, generations, generation_cutoff, adjacency_graph):
  state_data = state_generator(adjacency_graph);
  upper_limit = state_data[0].shape[0];
  lower_limit = 1;
  
  model_params = []
  gene_range = list(range(lower_limit, upper_limit))
  results = []  
  chromosomes = generate_population(population, state_data);

  for i in range(10):
    crossover_param = np.random.choice(gene_range);
    mutation_param = np.random.choice(gene_range);
    alive_param = np.random.choice(gene_range);
    params = (population, generation_cutoff, crossover_param, mutation_param, alive_param);
    model_params.append((params, state_data, chromosomes, True));
  for model_param in model_params:
    results.append(model(*model_param));
  
  
  res_index = find_best_result(results)
  params = (population, generations,  model_params[res_index][0][2],  model_params[res_index][0][3],  model_params[res_index][0][4]);

  return model(params, state_data, results[res_index], False);

In [126]:
def find_best_result(results):
  tests = []
  for result in results:
    opt_fitness = result[0][1];
    curr_res = []
    for i in range (len(result[0][0])):
      if opt_fitness == result[i][1]:
        curr_res.append(result[i])
      else: 
        break;
    new_res = set(tuple([tuple(i[0]), i[1] ]) for i in curr_res)
    tests.append(new_res)
  set_tuple = [];
  for test in tests:
   set_tuple.append(tuple(test));

  set_tuple = [t1[0] for t1 in set_tuple]
  for i in range(len(set_tuple)):
    set_tuple[i] = (*set_tuple[i], i);
  set_tuple.sort(key=lambda a: a[1]);
  
  return set_tuple[0][2];

In [4]:
adjacent_nodes = np.array([[0, 1, 0, 1, 0, 0, 0, 1, 0],
                             [0, 0, 1, 0, 1, 0, 0, 0, 0],
                             [0, 0, 0, 1, 0, 1, 0, 0, 1],
                             [1, 1, 0, 0, 0, 1, 1, 0, 0],
                             [0, 1, 0, 0, 0, 0, 0, 1, 0],
                             [0, 0, 1, 0, 0, 0, 1, 0, 1],
                             [0, 1, 0, 0, 1, 1, 0, 1, 0],
                             [1, 0, 0, 0, 0, 0, 0, 0, 1],
                             [0, 0, 0, 1, 0, 0, 0, 0, 1]])

In [ ]:
result = simulation(100, 1000, 200, adjacent_nodes);
result_to_csv((result[0][0],adjacent_nodes));